In [1]:
import numpy as np
import bilby
import pylab as plt
from gwsnr import GWSNR as snr

In [2]:
# nsamples is use for fitting. default: nsamples=100
# nsamples with higher value will give better result.
# for BNS events nsamples can be kept low
# with defult bilby's psd
snr_ = snr(waveform_approximant = 'IMRPhenomXPHM', snr_type = 'inner_product')

Given: IMR waveform, {self.waveform_approximant}.
psds not given. Choosing bilby's default psds
given psds:  {'L1': 'aLIGO_O4_high_asd.txt', 'H1': 'aLIGO_O4_high_asd.txt', 'V1': 'AdV_asd.txt'}


# bilby snr with injection

In [3]:
# check for all types of injection
injection_parameters = dict(mass_1=10.,
                            mass_2=10.,
                            luminosity_distance=100.0,
                            theta_jn=0.5,
                            psi=0.5,
                            phase=0.5,
                            geocent_time=1246527224.169434,
                            ra=0.0,
                            dec=0.0,
                            a_1=0.5,
                            a_2=0.5,
                            tilt_1=0.5,
                            tilt_2=0.5,
                            phi_12=0.5,
                            phi_jl=0.5,
                        )

In [4]:
def duration_fn(mass_1, mass_2, f_min):
    safety = 1.2
    approx_duration = safety * snr_.findchirp_chirptime(mass_1, mass_2, f_min)
    return np.ceil(approx_duration + 2.0)

In [5]:
sampling_frequency = 2048.
minimum_frequency = 20.
duration = duration_fn(mass_1=injection_parameters['mass_1'],
                       mass_2=injection_parameters['mass_2'],
                       f_min=20.)

waveform_arguments = dict(waveform_approximant = 'IMRPhenomXPHM',
                            reference_frequency = 20.,
                            minimum_frequency = minimum_frequency) 
waveform_generator = bilby.gw.WaveformGenerator(duration = duration,
    sampling_frequency = sampling_frequency,
    frequency_domain_source_model = bilby.gw.source.lal_binary_black_hole,
    parameters=injection_parameters,
    waveform_arguments = waveform_arguments
    )

19:07 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_black_hole
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters


In [6]:
# add kagra
asd_file = ['aLIGO_O4_high_asd.txt', 'aLIGO_O4_high_asd.txt', 'AdV_asd.txt']
ifos = bilby.gw.detector.InterferometerList(['H1', 'L1', 'V1'])

for i in range(3):
    ifos[i].power_spectral_density = bilby.gw.detector.PowerSpectralDensity(asd_file=asd_file[i])

ifos.set_strain_data_from_power_spectral_densities(
    sampling_frequency=sampling_frequency,
    duration=duration,
    start_time=injection_parameters["geocent_time"] + 2 - duration,
)

In [7]:
# make sure you getting right psd
print(ifos[0].power_spectral_density)

PowerSpectralDensity(psd_file='None', asd_file='/Users/hemantaph/anaconda3/envs/lal2/lib/python3.10/site-packages/bilby/gw/detector/noise_curves/aLIGO_O4_high_asd.txt')


In [8]:
bilby.core.utils.logger.disabled = True
ifos.inject_signal(
    waveform_generator=waveform_generator, parameters=injection_parameters
);

In [9]:
# from bilby injection
print(f"snr_L1 = {ifos[1].meta_data['optimal_SNR']}")
print(f"snr_H1 = {ifos[0].meta_data['optimal_SNR']}")
print(f"snr_V1 = {ifos[2].meta_data['optimal_SNR']}")
opt_snr = 0
for i in range(3):
    opt_snr += ifos[i].meta_data['optimal_SNR']**2
opt_snr = np.sqrt(opt_snr)
print(f"optimal snr = {opt_snr}")

snr_L1 = 117.4069218574123
snr_H1 = 74.60709879630028
snr_V1 = 34.995954604839696
optimal snr = 143.44100295779248


# inner product

In [55]:
# with inner_product with gwsnr package
# hp_inner_hc included in the package
snr_test = snr_.snr(mass_1=10.,
                    mass_2=10.,
                    luminosity_distance=100.0,
                    iota=0.5,
                    psi=0.5,
                    phase=0.5,
                    geocent_time=1246527224.169434,
                    ra=0.0,
                    dec=0.0,
                    a_1=0.5,
                    a_2=0.5,
                    tilt_1=0.5,
                    tilt_2=0.5,
                    phi_12=0.5,
                    phi_jl=0.5,
)
print(f"snr_L1 = {snr_test['L1']}")
print(f"snr_H1 = {snr_test['H1']}")
print(f"snr_V1 = {snr_test['V1']}")
print(f"optimal snr = {snr_test['opt_snr_net']}")

solving SNR with inner product


  0%|                                                                         | 0/1 [00:00<?, ?it/s]100%|█████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 27.09it/s]

snr_L1 = [117.59034621]
snr_H1 = [74.77707807]
snr_V1 = [35.24596307]
optimal snr = [143.74066523]


In [11]:
# with inner_product with gwsnr package
# hp_inner_hc included in the package
# note: I have made some changes in the code
# in the function noise_weighted_inner_prod of GWSNR class
snr_test = snr_.snr(mass_1=10.,
                    mass_2=10.,
                    luminosity_distance=100.0,
                    iota=0.5,
                    psi=0.5,
                    phase=0.5,
                    geocent_time=1246527224.169434,
                    ra=0.0,
                    dec=0.0,
                    a_1=0.5,
                    a_2=0.5,
                    tilt_1=0.5,
                    tilt_2=0.5,
                    phi_12=0.5,
                    phi_jl=0.5,
)
print(f"snr_L1 = {snr_test['L1']}")
print(f"snr_H1 = {snr_test['H1']}")
print(f"snr_V1 = {snr_test['V1']}")
print(f"snr_V1 = {snr_test['opt_snr_net']}")

solving SNR with inner product


  0%|                                                                         | 0/1 [00:00<?, ?it/s]100%|█████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 26.93it/s]

snr_L1 = [120.07165238]
snr_H1 = [74.69774127]
snr_V1 = [41.03722018]
snr_V1 = [147.24472044]


In [6]:
type(snr_.power_spectral_density_pycbc(psd='aLIGOaLIGODesignSensitivityT1800044'))

bilby.gw.detector.psd.PowerSpectralDensity

In [27]:
polas = waveform_generator.frequency_domain_strain(parameters=injection_parameters)
waveform = ifos[0].get_detector_response(waveform_polarizations=polas, parameters=injection_parameters)

In [28]:
waveform

array([ 0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j, ...,
       -1.92460287e-25-1.25518076e-25j,
       -8.96903650e-27-6.66481015e-27j,
       -0.00000000e+00+0.00000000e+00j])

In [29]:
polas

{'plus': array([ 0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j, ...,
         6.30108866e-26-4.41912624e-25j,
        -1.91315748e-26-1.00891996e-26j,
         0.00000000e+00+0.00000000e+00j]),
 'cross': array([ 0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j, ...,
        -4.42943584e-25-6.15895824e-26j,
        -1.05659226e-26+1.89869550e-26j,
         0.00000000e+00+0.00000000e+00j])}